In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd  # Biblioteca para manipulação de dados em formato tabular
import tensorflow as tf  # Biblioteca para construção e treinamento de modelos de deep learning
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, LeakyReLU  # Camadas da Keras para construir a arquitetura do modelo
from tensorflow.keras.models import Model  # Classe para definir o modelo funcional da Keras
from tensorflow.keras.callbacks import EarlyStopping  # Callback para parar o treinamento quando não houver melhora
from sklearn.model_selection import train_test_split  # Função para dividir o dataset em conjuntos de treino e teste
from sklearn.preprocessing import MinMaxScaler  # Classe para normalizar dados entre um intervalo
import numpy as np  # Biblioteca para manipulação de arrays e cálculos numéricos
import random  # Biblioteca para gerar valores aleatórios e definir seeds
import motmetrics as mm  # Biblioteca para calcular métricas de rastreamento
from scipy.optimize import linear_sum_assignment  # Função para resolver o problema de atribuição linear
from scipy.spatial.distance import cdist  # Função para calcular a distância entre vetores em espaços de múltiplas dimensões


#Modelo com 2 branch:
1 - input de coodenadas X frames t-1 e t.<br>
2 - input de coordenadas Y frames t-1 e t.

##Preparação de Ambiente e Dados

In [ ]:
# Definir seed para reprodutibilidade
SEED = 42  # Define uma constante para a seed
np.random.seed(SEED)  # Define a seed para o NumPy, garantindo resultados consistentes em operações aleatórias
tf.random.set_seed(SEED)  # Define a seed para operações aleatórias do TensorFlow, garantindo reprodutibilidade nos modelos
random.seed(SEED)  # Define a seed para o módulo random do Python, para garantir consistência em operações de aleatoriedade gerais


In [ ]:
# Carregar o arquivo CSV
path_filtered = 'https://drive.google.com/uc?id=1PBBHU8JvnOBBVeV0HcyLjcxyR8upZafX'  # Define o caminho do arquivo CSV no Google Drive
data = pd.read_csv(path_filtered)  # Carrega o arquivo CSV no DataFrame 'data' usando a função read_csv do Pandas


###Função para preparar os dados

In [ ]:
# Função para preparar os dados para predição
def prepare_data_for_prediction(df):
    # Ordena o DataFrame por instance_token e timestamp para garantir a sequência correta dos dados temporais
    df = df.sort_values(by=['instance_token', 'timestamp']).reset_index(drop=True)

    # Filtra apenas as colunas relevantes para análise
    df = df[['instance_token', 'timestamp', 'camera', 'bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y', 'scene_token']]

    # Cria vetores para as coordenadas x e y da bounding box
    df['bbox_vector_x'] = df.apply(lambda row: [row['bbox_min_x'], row['bbox_max_x']], axis=1)  # Vetor com limites mínimo e máximo no eixo x
    df['bbox_vector_y'] = df.apply(lambda row: [row['bbox_min_y'], row['bbox_max_y']], axis=1)  # Vetor com limites mínimo e máximo no eixo y

    # Inicializa listas para armazenar inputs e outputs
    inputs_x_t_minus_1 = []  # Armazena coordenadas x no tempo t-1
    inputs_y_t_minus_1 = []  # Armazena coordenadas y no tempo t-1
    inputs_x_t = []  # Armazena coordenadas x no tempo t
    inputs_y_t = []  # Armazena coordenadas y no tempo t
    outputs_x = []  # Armazena coordenadas x no tempo t+1
    outputs_y = []  # Armazena coordenadas y no tempo t+1
    metadata_list = []  # Armazena informações sobre a instância, câmera e tempo

    # Obtém uma lista única de instâncias
    unique_instances = df['instance_token'].unique()

    for instance in unique_instances:
        instance_data = df[df['instance_token'] == instance].reset_index(drop=True)  # Filtra dados para a instância atual e reseta o índice
        instance_samples_x = instance_data['bbox_vector_x'].values  # Extrai os vetores de coordenadas x
        instance_samples_y = instance_data['bbox_vector_y'].values  # Extrai os vetores de coordenadas y
        instance_timestamps = instance_data['timestamp'].values  # Extrai timestamps
        instance_cameras = instance_data['camera'].values  # Extrai dados de câmera
        scene_tokens = instance_data['scene_token'].values  # Extrai dados de cena

        num_samples = len(instance_samples_x)  # Obtém o número de amostras para a instância atual
        for i in range(1, num_samples - 1):
            # Verifica se as amostras atuais e a próxima estão na mesma câmera
            if instance_cameras[i - 1] != instance_cameras[i] or instance_cameras[i] != instance_cameras[i + 1]:
                continue  # Ignora se há uma mudança de câmera entre as amostras

            # Utiliza as bounding boxes nos tempos t-1 e t como input e a no tempo t+1 como output
            inputs_x_t_minus_1.append(instance_samples_x[i - 1])
            inputs_y_t_minus_1.append(instance_samples_y[i - 1])
            inputs_x_t.append(instance_samples_x[i])
            inputs_y_t.append(instance_samples_y[i])
            outputs_x.append(instance_samples_x[i + 1])
            outputs_y.append(instance_samples_y[i + 1])
            metadata_list.append({
                'instance_token': instance,
                'timestamp': instance_timestamps[i + 1],
                'camera': instance_cameras[i + 1],
                'scene_token': scene_tokens[i + 1]
            })  # Armazena metadados relevantes para cada sequência

    # Converte listas para arrays NumPy
    inputs_x_t_minus_1 = np.array(inputs_x_t_minus_1)
    inputs_y_t_minus_1 = np.array(inputs_y_t_minus_1)
    inputs_x_t = np.array(inputs_x_t)
    inputs_y_t = np.array(inputs_y_t)
    outputs_x = np.array(outputs_x)
    outputs_y = np.array(outputs_y)

    # Converte metadata_list para um DataFrame
    metadata = pd.DataFrame(metadata_list)

    # Verifica a consistência dos comprimentos das listas
    assert len(inputs_x_t_minus_1) == len(inputs_y_t_minus_1) == len(inputs_x_t) == len(inputs_y_t) == len(outputs_x) == len(outputs_y) == len(metadata), "Inconsistência nos comprimentos das listas de saída."

    return inputs_x_t_minus_1, inputs_y_t_minus_1, inputs_x_t, inputs_y_t, outputs_x, outputs_y, metadata  # Retorna inputs e outputs preparados junto com metadados


###Separação de Dados de Treino e Teste

In [ ]:
# Dividir os dados em treino e teste garantindo que cenas não sejam misturadas
unique_scenes = data['scene_token'].unique()  # Obtém uma lista única de cenas no dataset
train_scenes, test_scenes = train_test_split(unique_scenes, test_size=0.2, random_state=SEED)  # Divide as cenas em 80% para treino e 20% para teste

# Separa o conjunto de treino e teste garantindo que cada conjunto contenha cenas distintas
train_data = data[data['scene_token'].isin(train_scenes)].reset_index(drop=True)  # Filtra o dataset para o conjunto de treino e reseta o índice
test_data = data[data['scene_token'].isin(test_scenes)].reset_index(drop=True)  # Filtra o dataset para o conjunto de teste e reseta o índice

# Preparar os dados de treino e teste
train_inputs_x_t_minus_1, train_inputs_y_t_minus_1, train_inputs_x_t, train_inputs_y_t, train_outputs_x, train_outputs_y, _ = prepare_data_for_prediction(train_data)  # Prepara os dados de treino chamando a função de preparação
test_inputs_x_t_minus_1, test_inputs_y_t_minus_1, test_inputs_x_t, test_inputs_y_t, test_outputs_x, test_outputs_y, test_metadata = prepare_data_for_prediction(test_data)  # Prepara os dados de teste chamando a função de preparação


##Testes de combinações de hiperparâmetros

###Função para criar o modelo

In [ ]:
# Aplicar normalização aos dados
scaler = MinMaxScaler()  # Inicializa o objeto MinMaxScaler para normalizar os dados entre 0 e 1

# Empilha todas as entradas e saídas de treinamento
train_data_combined = np.vstack([
    train_inputs_x_t_minus_1,
    train_inputs_y_t_minus_1,
    train_inputs_x_t,
    train_inputs_y_t,
    train_outputs_x,
    train_outputs_y
])  # Combina todos os dados de treino em uma única matriz para ajustar o scaler

# Ajusta o scaler nos dados combinados
scaler.fit(train_data_combined)  # Ajusta o scaler com base nos dados de treino combinados


# Normaliza os dados de treino
train_inputs_x_t_minus_1 = scaler.transform(train_inputs_x_t_minus_1.reshape(-1, 2)).reshape(-1, 2)
train_inputs_y_t_minus_1 = scaler.transform(train_inputs_y_t_minus_1.reshape(-1, 2)).reshape(-1, 2)
train_inputs_x_t = scaler.transform(train_inputs_x_t.reshape(-1, 2)).reshape(-1, 2)
train_inputs_y_t = scaler.transform(train_inputs_y_t.reshape(-1, 2)).reshape(-1, 2)
train_outputs_x = scaler.transform(train_outputs_x.reshape(-1, 2)).reshape(-1, 2)
train_outputs_y = scaler.transform(train_outputs_y.reshape(-1, 2)).reshape(-1, 2)

# Normaliza os dados de teste
test_inputs_x_t_minus_1 = scaler.transform(test_inputs_x_t_minus_1.reshape(-1, 2)).reshape(-1, 2)
test_inputs_y_t_minus_1 = scaler.transform(test_inputs_y_t_minus_1.reshape(-1, 2)).reshape(-1, 2)
test_inputs_x_t = scaler.transform(test_inputs_x_t.reshape(-1, 2)).reshape(-1, 2)
test_inputs_y_t = scaler.transform(test_inputs_y_t.reshape(-1, 2)).reshape(-1, 2)
test_outputs_x = scaler.transform(test_outputs_x.reshape(-1, 2)).reshape(-1, 2)
test_outputs_y = scaler.transform(test_outputs_y.reshape(-1, 2)).reshape(-1, 2)


# Função para criar o modelo siamês com dois branches separados para coordenadas X e Y
def create_siamese_nn(input_shape, units, dropout_rate, activation):
    # Define os inputs para as coordenadas X e Y
    input_x_t_minus_1 = Input(shape=input_shape, name='input_x_t_minus_1')
    input_x_t = Input(shape=input_shape, name='input_x_t')
    input_y_t_minus_1 = Input(shape=input_shape, name='input_y_t_minus_1')
    input_y_t = Input(shape=input_shape, name='input_y_t')

    # Rede compartilhada para coordenadas X
    def shared_network_x(input_layer):
        x = Dense(units, activation=activation)(input_layer)  # Camada densa com ativação definida
        x = Dropout(dropout_rate)(x)  # Aplica Dropout para regularização
        x = Dense(units, activation=activation)(x)  # Outra camada densa com ativação
        return x  # Retorna o resultado final para o branch X

    # Rede compartilhada para coordenadas Y
    def shared_network_y(input_layer):
        y = Dense(units, activation=activation)(input_layer)  # Camada densa com ativação definida
        y = Dropout(dropout_rate)(y)  # Aplica Dropout para regularização
        y = Dense(units, activation=activation)(y)  # Outra camada densa com ativação
        return y  # Retorna o resultado final para o branch Y

    # Aplica as redes compartilhadas aos inputs
    branch_x_t_minus_1 = shared_network_x(input_x_t_minus_1)
    branch_x_t = shared_network_x(input_x_t)
    branch_y_t_minus_1 = shared_network_y(input_y_t_minus_1)
    branch_y_t = shared_network_y(input_y_t)

    # Concatena as saídas dos branches X e Y separadamente
    concatenated_x = Concatenate()([branch_x_t_minus_1, branch_x_t])  # Concatena as saídas para as coordenadas X
    concatenated_y = Concatenate()([branch_y_t_minus_1, branch_y_t])  # Concatena as saídas para as coordenadas Y

    # Camadas adicionais para previsão de X e Y
    x = Dense(64, activation=activation)(concatenated_x)  # Camada densa para previsão das coordenadas X
    output_x = Dense(2, activation='linear')(x)  # Camada de saída linear para previsão das coordenadas X futuras (min_x, max_x)

    y = Dense(64, activation=activation)(concatenated_y)  # Camada densa para previsão das coordenadas Y
    output_y = Dense(2, activation='linear')(y)  # Camada de saída linear para previsão das coordenadas Y futuras (min_y, max_y)

    # Retorna o modelo siamês final com os inputs e outputs definidos
    return Model(inputs=[input_x_t_minus_1, input_x_t, input_y_t_minus_1, input_y_t], outputs=[output_x, output_y])


###Hiperparâmetros a serem testados

In [ ]:
# Hiperparâmetros a serem explorados
units_list = [32, 64, 128, 256, 512, 1024]  # Lista de diferentes números de unidades para camadas densas
dropout_rates = [0.1, 0.2]  # Taxas de dropout para regularização, evitando overfitting
epochs_list = [10, 20, 30, 40, 50]  # Número de épocas para o treinamento do modelo
activations = ['relu', 'linear', 'selu', 'swish', 'mish', LeakyReLU()]  # Funções de ativação para as camadas, influenciando como os valores são transformados


###Iteração para tester todas as combinações de hiperparâmetros

In [ ]:
from tensorflow.keras.utils import get_custom_objects

# Registrar a função de ativação mish manualmente
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))  # Define a função mish como uma combinação de tanh e softplus

get_custom_objects().update({'mish': mish})  # Registra a função mish como ativação personalizada


# Função para calcular o IoU de forma vetorizada
def calcular_iou_vetorizado(bbox_real, bbox_predito):
    xA = np.maximum(bbox_real[:, 0], bbox_predito[:, 0])  # Calcula o ponto x mais à direita da interseção
    yA = np.maximum(bbox_real[:, 2], bbox_predito[:, 2])  # Calcula o ponto y mais alto da interseção
    xB = np.minimum(bbox_real[:, 1], bbox_predito[:, 1])  # Calcula o ponto x mais à esquerda da interseção
    yB = np.minimum(bbox_real[:, 3], bbox_predito[:, 3])  # Calcula o ponto y mais baixo da interseção

    interArea = np.maximum(0, xB - xA) * np.maximum(0, yB - yA)  # Calcula a área de interseção das bounding boxes
    boxA_area = (bbox_real[:, 1] - bbox_real[:, 0]) * (bbox_real[:, 3] - bbox_real[:, 2])  # Área da bbox real
    boxB_area = (bbox_predito[:, 1] - bbox_predito[:, 0]) * (bbox_predito[:, 3] - bbox_predito[:, 2])  # Área da bbox predita

    epsilon = 1e-7  # Para evitar divisão por zero
    iou = interArea / (boxA_area + boxB_area - interArea + epsilon)  # Calcula o IoU
    return iou


best_models = []  # Lista para armazenar os modelos com os melhores IoUs

# Loop sobre os hiperparâmetros para testar diferentes configurações do modelo siamês
for units in units_list:
    for dropout_rate in dropout_rates:
        for epochs in epochs_list:
            for activation in activations:
                activation_name = activation if isinstance(activation, str) else activation.__class__.__name__  # Obtém o nome da ativação
                print(f'Treinando modelo siamês com units={units}, dropout_rate={dropout_rate}, epochs={epochs}, activation={activation_name}')

                # Cria o modelo siamês com a configuração de hiperparâmetros atual
                siamese_model = create_siamese_nn((2,), units, dropout_rate, activation)
                siamese_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])  # Compila o modelo com função de perda MSE

                # Implementa Early Stopping com min_delta para interromper o treinamento quando não houver melhora
                early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, min_delta=0.001)
                siamese_model.fit(
                    [train_inputs_x_t_minus_1, train_inputs_x_t, train_inputs_y_t_minus_1, train_inputs_y_t],
                    [train_outputs_x, train_outputs_y],
                    batch_size=32, epochs=epochs,
                    validation_data=([
                        test_inputs_x_t_minus_1, test_inputs_x_t,
                        test_inputs_y_t_minus_1, test_inputs_y_t
                    ], [test_outputs_x, test_outputs_y]),
                    callbacks=[early_stopping],
                    verbose=0  # Altere verbose=1 para acompanhar o progresso do treinamento
                )

                # Fazer previsões no conjunto de teste
                best_predictions_x, best_predictions_y = siamese_model.predict([
                    test_inputs_x_t_minus_1, test_inputs_x_t, test_inputs_y_t_minus_1, test_inputs_y_t
                ])

                # Desfazer a normalização para calcular o IoU
                best_predictions_x = scaler.inverse_transform(best_predictions_x.reshape(-1, 2)).reshape(-1, 2)
                best_predictions_y = scaler.inverse_transform(best_predictions_y.reshape(-1, 2)).reshape(-1, 2)
                test_outputs_x_unscaled = scaler.inverse_transform(test_outputs_x.reshape(-1, 2)).reshape(-1, 2)
                test_outputs_y_unscaled = scaler.inverse_transform(test_outputs_y.reshape(-1, 2)).reshape(-1, 2)

                # Ajustar o comprimento de test_metadata para coincidir com o das predições
                correct_length = len(test_outputs_x_unscaled)
                if len(test_metadata) > correct_length:
                    print(f"Ajustando test_metadata de {len(test_metadata)} para {correct_length} entradas.")
                    test_metadata = test_metadata[:correct_length]
                elif len(test_metadata) < correct_length:
                    print(f"Advertência: test_metadata possui {len(test_metadata)}, mas as predições possuem {correct_length}. Ajustando predições.")
                    # Trunca as predições para coincidir com test_metadata
                    best_predictions_x = best_predictions_x[:len(test_metadata)]
                    best_predictions_y = best_predictions_y[:len(test_metadata)]
                    test_outputs_x_unscaled = test_outputs_x_unscaled[:len(test_metadata)]
                    test_outputs_y_unscaled = test_outputs_y_unscaled[:len(test_metadata)]
                    correct_length = len(test_metadata)

                # Cria DataFrames para cálculo do IoU
                bboxes_real = np.hstack((test_outputs_x_unscaled, test_outputs_y_unscaled))  # Real
                bboxes_pred = np.hstack((best_predictions_x, best_predictions_y))  # Predito
                ious = calcular_iou_vetorizado(bboxes_real, bboxes_pred)  # Calcula IoU para cada amostra
                mean_iou = np.mean(ious)  # Calcula o IoU médio
                print(f'IoU Médio do modelo siamês: {mean_iou:.2f}')

                # Armazena o modelo e suas métricas na lista best_models
                best_models.append({
                    'model': siamese_model,
                    'iou': mean_iou,
                    'config': {
                        'units': units,
                        'dropout_rate': dropout_rate,
                        'epochs': epochs,
                        'activation_name': activation_name
                    }
                })

# Ordena os modelos pelo maior IoU e mantém os 10 melhores
best_models = sorted(best_models, key=lambda x: x['iou'], reverse=True)[:10]


Treinando modelo siamês com units=32, dropout_rate=0.1, epochs=10, activation=relu
IoU Médio do modelo siamês: 0.03
Treinando modelo siamês com units=32, dropout_rate=0.1, epochs=10, activation=linear
IoU Médio do modelo siamês: 0.09
Treinando modelo siamês com units=32, dropout_rate=0.1, epochs=10, activation=selu
IoU Médio do modelo siamês: 0.10
Treinando modelo siamês com units=32, dropout_rate=0.1, epochs=10, activation=swish
IoU Médio do modelo siamês: 0.18
Treinando modelo siamês com units=32, dropout_rate=0.1, epochs=10, activation=mish
IoU Médio do modelo siamês: 0.16
Treinando modelo siamês com units=32, dropout_rate=0.1, epochs=10, activation=LeakyReLU
IoU Médio do modelo siamês: 0.05
Treinando modelo siamês com units=32, dropout_rate=0.1, epochs=20, activation=relu
IoU Médio do modelo siamês: 0.07
Treinando modelo siamês com units=32, dropout_rate=0.1, epochs=20, activation=linear
IoU Médio do modelo siamês: 0.23
Treinando modelo siamês com units=32, dropout_rate=0.1, epochs

In [ ]:
def analyze_bbox_movements(df):
    # Ordena o DataFrame por instance_token, camera e timestamp para garantir a ordem correta
    df = df.sort_values(by=['instance_token', 'camera', 'timestamp']).reset_index(drop=True)
    distances = []  # Inicializa uma lista para armazenar as distâncias

    unique_instances = df['instance_token'].unique()  # Obtém todas as instâncias únicas de objetos

    # Itera por cada instância
    for instance in unique_instances:
        instance_data = df[df['instance_token'] == instance].reset_index(drop=True)  # Filtra dados da instância atual
        cameras = instance_data['camera'].unique()  # Obtém todas as câmeras usadas para a instância

        # Itera por cada câmera associada à instância
        for cam in cameras:
            cam_data = instance_data[instance_data['camera'] == cam].reset_index(drop=True)  # Filtra dados da câmera atual
            bboxes = cam_data[['bbox_min_x', 'bbox_min_y', 'bbox_max_x', 'bbox_max_y']].values  # Extrai coordenadas da bounding box

            # Itera sobre cada bounding box e calcula a distância entre centros consecutivos
            for i in range(1, len(bboxes)):
                bbox_prev = bboxes[i - 1]  # Bounding box anterior
                bbox_curr = bboxes[i]  # Bounding box atual

                # Calcula os centros das bounding boxes anterior e atual
                center_prev = [(bbox_prev[0] + bbox_prev[2]) / 2, (bbox_prev[1] + bbox_prev[3]) / 2]
                center_curr = [(bbox_curr[0] + bbox_curr[2]) / 2, (bbox_curr[1] + bbox_curr[3]) / 2]

                # Calcula a distância euclidiana entre os centros
                dist = np.linalg.norm(np.array(center_curr) - np.array(center_prev))
                distances.append(dist)  # Adiciona a distância calculada à lista

    return distances  # Retorna a lista de distâncias


# Calcula as distâncias no conjunto de dados de treino
distances = analyze_bbox_movements(train_data)

# Converte as distâncias para um array NumPy para facilitar operações matemáticas
distances = np.array(distances)

# Calcula resumos estatísticos das distâncias
mean_distance = np.mean(distances)  # Média das distâncias
median_distance = np.median(distances)  # Mediana das distâncias
std_distance = np.std(distances)  # Desvio padrão das distâncias
min_distance = np.min(distances)  # Menor distância
max_distance = np.max(distances)  # Maior distância

# Exibe os resumos estatísticos
print("Resumos Estatísticos dos Movimentos das Bounding Boxes:")
print(f"Distância Média: {mean_distance:.2f}")
print(f"Mediana da Distância: {median_distance:.2f}")
print(f"Desvio Padrão: {std_distance:.2f}")
print(f"Distância Mínima: {min_distance:.2f}")
print(f"Distância Máxima: {max_distance:.2f}")


Statistical Summaries of Bounding Box Movements:
Mean Distance: 103.28
Median Distance: 58.08
Standard Deviation: 125.89
Minimum Distance: 0.00
Maximum Distance: 1215.51


In [ ]:
# Imprime as 10 melhores configurações encontradas
for idx, best_model in enumerate(best_models, start=1):
    print(f'Modelo {idx}:')
    print(f'Units: {best_model["config"]["units"]}')  # Exibe a quantidade de unidades na camada oculta
    print(f'Dropout Rate: {best_model["config"]["dropout_rate"]}')  # Exibe a taxa de dropout usada
    print(f'Epochs: {best_model["config"]["epochs"]}')  # Exibe o número de épocas de treinamento
    print(f'Activation Name: {best_model["config"]["activation_name"]}')  # Exibe a função de ativação
    print(f'IoU: {best_model["iou"]:.2f}\n')  # Exibe a média do IoU obtida pelo modelo

# Função de rastreamento simples
def simple_tracker(pred_bboxes, timestamps, cameras, max_distance=median_distance):
    pred_ids_array = np.zeros(len(pred_bboxes), dtype=int)  # Inicializa array para IDs preditos
    next_id = 1  # Inicializa o ID a ser atribuído para o próximo objeto novo
    active_tracks_per_camera = {}  # Dicionário para armazenar os tracks ativos por câmera

    # Inicializa tracks ativos para cada câmera
    for cam in np.unique(cameras):
        active_tracks_per_camera[cam] = {}

    # Ordena índices para processar os dados na ordem de timestamp e câmera
    sort_idx = np.lexsort((timestamps, cameras))
    pred_bboxes_sorted = pred_bboxes[sort_idx]  # Ordena as bounding boxes com base nos índices
    timestamps_sorted = timestamps[sort_idx]  # Ordena os timestamps com base nos índices
    cameras_sorted = cameras[sort_idx]  # Ordena as câmeras com base nos índices

    # Itera sobre cada bounding box predita
    for idx in range(len(pred_bboxes_sorted)):
        cam = cameras_sorted[idx]  # Obtém a câmera atual
        t = timestamps_sorted[idx]  # Obtém o timestamp atual
        bbox = pred_bboxes_sorted[idx]  # Obtém a bounding box atual
        idx_original = sort_idx[idx]  # Recupera o índice original

        active_tracks = active_tracks_per_camera[cam]  # Tracks ativos para a câmera atual

        # Cria um novo track se não houver ativos para a câmera
        if not active_tracks:
            pred_ids_array[idx_original] = next_id
            active_tracks[next_id] = {'bbox': bbox, 'timestamp': t}
            next_id += 1  # Incrementa o próximo ID
        else:
            # Calcula distâncias entre tracks ativos e a bounding box atual
            track_ids = list(active_tracks.keys())
            track_bboxes = np.array([active_tracks[tid]['bbox'] for tid in track_ids])

            distances = cdist(track_bboxes, [bbox], metric='euclidean').flatten()  # Calcula distância euclidiana
            min_dist_idx = np.argmin(distances)  # Índice do track mais próximo
            min_dist = distances[min_dist_idx]  # Distância mínima

            # Verifica se a distância mínima é menor que a distância máxima permitida
            if min_dist < max_distance:
                tid = track_ids[min_dist_idx]  # Obtém o ID do track mais próximo
                pred_ids_array[idx_original] = tid
                active_tracks[tid] = {'bbox': bbox, 'timestamp': t}  # Atualiza o track ativo
            else:
                # Cria um novo track se nenhum existente estiver próximo
                pred_ids_array[idx_original] = next_id
                active_tracks[next_id] = {'bbox': bbox, 'timestamp': t}
                next_id += 1  # Incrementa o próximo ID

    return pred_ids_array  # Retorna o array com os IDs preditos


Modelo 1:
Units: 128
Dropout Rate: 0.1
Epochs: 20
Activation Name: linear
IoU: 0.34

Modelo 2:
Units: 64
Dropout Rate: 0.1
Epochs: 20
Activation Name: mish
IoU: 0.33

Modelo 3:
Units: 512
Dropout Rate: 0.2
Epochs: 30
Activation Name: linear
IoU: 0.33

Modelo 4:
Units: 512
Dropout Rate: 0.1
Epochs: 50
Activation Name: mish
IoU: 0.32

Modelo 5:
Units: 256
Dropout Rate: 0.2
Epochs: 10
Activation Name: swish
IoU: 0.32

Modelo 6:
Units: 1024
Dropout Rate: 0.1
Epochs: 50
Activation Name: linear
IoU: 0.32

Modelo 7:
Units: 256
Dropout Rate: 0.1
Epochs: 30
Activation Name: linear
IoU: 0.32

Modelo 8:
Units: 1024
Dropout Rate: 0.1
Epochs: 20
Activation Name: LeakyReLU
IoU: 0.31

Modelo 9:
Units: 128
Dropout Rate: 0.1
Epochs: 10
Activation Name: swish
IoU: 0.31

Modelo 10:
Units: 1024
Dropout Rate: 0.1
Epochs: 10
Activation Name: selu
IoU: 0.31



In [ ]:
def apply_mot_metrics(df):
    # Cria um identificador único para cada combinação de instance_token e camera
    df['gt_id'] = df.apply(lambda row: f"{row['instance_token']}_{row['camera']}", axis=1)

    # Mapeia cada combinação única para um ID numérico único
    unique_gt_ids = df['gt_id'].unique()
    id_mapping_gt = {token: idx + 1 for idx, token in enumerate(unique_gt_ids)}  # Inicia IDs a partir de 1
    df['gt_id_num'] = df['gt_id'].map(id_mapping_gt)

    # Inicializa o acumulador de métricas com auto_id=False para evitar IDs automáticos
    acc = mm.MOTAccumulator(auto_id=False)

    # Atribui um frameid sequencial para cada timestamp único, garantindo que os IDs de frame sejam únicos
    unique_timestamps = sorted(df['timestamp'].unique())
    timestamp_to_frameid = {timestamp: idx for idx, timestamp in enumerate(unique_timestamps)}
    df['frameid'] = df['timestamp'].map(timestamp_to_frameid)

    # Ordena o DataFrame por frameid
    df = df.sort_values(by=['frameid'])

    # Agrupa as detecções por frameid
    grouped = df.groupby('frameid')

    for frameid, group in grouped:
        # IDs e bounding boxes de ground truth
        gt_ids = group['gt_id_num'].tolist()
        gt_bboxes = group[['true_min_x', 'true_min_y', 'true_max_x', 'true_max_y']].values

        # IDs e bounding boxes preditos
        pred_ids_frame = group['pred_id'].tolist()
        pred_bboxes_frame = group[['pred_min_x', 'pred_min_y', 'pred_max_x', 'pred_max_y']].values

        # Converte IDs para strings
        gt_ids = [str(id) for id in gt_ids]
        pred_ids_frame = [str(id) for id in pred_ids_frame]

        # Calcula a matriz de distâncias (usando IoU) para comparar as bounding boxes
        distances = mm.distances.iou_matrix(gt_bboxes, pred_bboxes_frame, max_iou=0.5)

        # Verifica se há gt_id duplicados no mesmo frameid e remove os duplicados se necessário deixando-os únicos
        duplicates = group.duplicated(subset=['gt_id_num'])
        if duplicates.any():
            print(f"Removendo {duplicates.sum()} duplicatas de gt_id no frameid {frameid}.")
            group = group.drop_duplicates(subset=['gt_id_num'], keep='first')
            gt_ids = group['gt_id_num'].tolist()
            gt_bboxes = group[['true_min_x', 'true_min_y', 'true_max_x', 'true_max_y']].values
            distances = mm.distances.iou_matrix(gt_bboxes, pred_bboxes_frame, max_iou=0.5)

        # Atualiza o acumulador com as detecções deste frame
        acc.update(
            gt_ids,
            pred_ids_frame,
            distances,
            frameid=frameid
        )

    return acc  # Retorna o acumulador preenchido para análise de métricas


metrics_results = []  # Lista para armazenar resultados de métricas de cada modelo

# Loop sobre os melhores modelos
for idx, best_model in enumerate(best_models, start=1):
    print(f'Processando Métricas para Modelo {idx}...')

    # Faz previsões com o modelo atual
    best_predictions_x, best_predictions_y = best_model['model'].predict([
        test_inputs_x_t_minus_1, test_inputs_x_t, test_inputs_y_t_minus_1, test_inputs_y_t
    ])

    # Desfaz a normalização para calcular métricas
    best_predictions_x = scaler.inverse_transform(best_predictions_x.reshape(-1, 2)).reshape(-1, 2)
    best_predictions_y = scaler.inverse_transform(best_predictions_y.reshape(-1, 2)).reshape(-1, 2)
    test_outputs_x_unscaled = scaler.inverse_transform(test_outputs_x.reshape(-1, 2)).reshape(-1, 2)
    test_outputs_y_unscaled = scaler.inverse_transform(test_outputs_y.reshape(-1, 2)).reshape(-1, 2)

    # Garante que test_metadata tenha o mesmo comprimento que as predições
    correct_length = len(test_outputs_x_unscaled)
    if len(test_metadata) > correct_length:
        print(f"Ajustando test_metadata de {len(test_metadata)} para {correct_length} entradas.")
        test_metadata = test_metadata.iloc[:correct_length]
    elif len(test_metadata) < correct_length:
        print(f"Advertência: test_metadata possui {len(test_metadata)}, mas as predições possuem {correct_length}. Ajustando predições.")
        best_predictions_x = best_predictions_x[:len(test_metadata)]
        best_predictions_y = best_predictions_y[:len(test_metadata)]
        test_outputs_x_unscaled = test_outputs_x_unscaled[:len(test_metadata)]
        test_outputs_y_unscaled = test_outputs_y_unscaled[:len(test_metadata)]
        correct_length = len(test_metadata)

    # Cria DataFrame com as informações para calcular métricas
    df_results = pd.DataFrame({
        'instance_token': test_metadata['instance_token'],
        'timestamp': test_metadata['timestamp'],
        'camera': test_metadata['camera'],
        'scene_token': test_metadata['scene_token'],
        'true_min_x': test_outputs_x_unscaled[:, 0],
        'true_max_x': test_outputs_x_unscaled[:, 1],
        'true_min_y': test_outputs_y_unscaled[:, 0],
        'true_max_y': test_outputs_y_unscaled[:, 1],
        'pred_min_x': best_predictions_x[:, 0],
        'pred_max_x': best_predictions_x[:, 1],
        'pred_min_y': best_predictions_y[:, 0],
        'pred_max_y': best_predictions_y[:, 1]
    })

    # Define timestamps e bounding boxes preditas para rastreamento
    df_results['timestamp'] = df_results['timestamp'].astype(int)
    pred_bboxes = df_results[['pred_min_x', 'pred_min_y', 'pred_max_x', 'pred_max_y']].values
    timestamps = df_results['timestamp'].values
    cameras = df_results['camera'].values

    # Aplica o tracker para atribuir IDs preditos
    pred_ids = simple_tracker(pred_bboxes, timestamps, cameras)
    df_results['pred_id'] = pred_ids

    # Aplica as métricas
    accumulator = apply_mot_metrics(df_results)

    # Cria o manipulador de métricas para gerar os resultados do acumulador
    mh = mm.metrics.create()

    # Calcula métricas específicas
    summary = mh.compute(
        accumulator,
        metrics=['num_frames', 'num_switches', 'mota', 'motp',  'idf1', 'idp', 'idf1', 'idp', 'idr', 'recall', 'precision'],
        name=f'Model_{idx}'
    )

    # Adiciona o resultado ao conjunto de métricas
    metrics_results.append(summary)

    print(f'Métricas calculadas para Modelo {idx}.\n')

# Se há resultados de métricas, salva-os em um arquivo CSV
if metrics_results:
    metrics_summary_df = pd.concat(metrics_results, axis=0).reset_index(drop=True)
    metrics_summary_df.to_csv('mot_metrics_summary.csv', index=False)
    print('Métricas MOT salvas em mot_metrics_summary.csv')
else:
    print('Nenhuma métrica calculada para os modelos selecionados.')

# Imprime o resumo das métricas
print(metrics_summary_df)


Processando Métricas para Modelo 1...
Métricas calculadas para Modelo 1.

Processando Métricas para Modelo 2...
Métricas calculadas para Modelo 2.

Processando Métricas para Modelo 3...
Métricas calculadas para Modelo 3.

Processando Métricas para Modelo 4...
Métricas calculadas para Modelo 4.

Processando Métricas para Modelo 5...
Métricas calculadas para Modelo 5.

Processando Métricas para Modelo 6...
Métricas calculadas para Modelo 6.

Processando Métricas para Modelo 7...
Métricas calculadas para Modelo 7.

Processando Métricas para Modelo 8...
Métricas calculadas para Modelo 8.

Processando Métricas para Modelo 9...
Métricas calculadas para Modelo 9.

Processando Métricas para Modelo 10...
Métricas calculadas para Modelo 10.

Métricas MOT salvas em mot_metrics_summary.csv
   num_frames  num_switches      mota      motp      idf1       idp       idr  \
0          76           475  0.588636  0.178857  0.692424  0.880539  0.692424   
1          76           477  0.582576  0.179085  